résumé de ce que j'ai fait:
génerer des données (sans le calcules de Moyenne Eligible Décision M E D) et sauvgarder en fichier data.txt
calcules de M E D sur un étudiant particulier
inserer les valeurs dans une ligne sur notre fichier (je pourrai eveuellement travailler sur le dataframe directement)


In [57]:
import pandas as pd
import numpy as np

# pour la partie 3
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report

N_exemplaire = 100
file = "data"
arrond = 0
coeffs= [1, 1, 2, 0, 1] #calcul de la Moyenne
seuils = [11, 9.2, 11.23] # Calcul du status d'éligibilité (logique, BDD, PS)
thresholds = [12, 14] # prendre la décision

# 1 - Data generation

In [58]:
def generer_donnees(N_exemplaire=N_exemplaire, fichier=file, arrond=arrond):
    means = np.random.uniform(9, 15, size=8)
    stds = np.random.uniform(1, 3, size=8)
    column_names = ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']

    data = {}
    data['N_candidat'] = range(0, N_exemplaire)
    for i, col_name in enumerate(column_names):
        values = np.random.normal(means[i], stds[i], N_exemplaire)
        if arrond > 0:
            values = np.round(values, arrond)
        elif arrond<0:
            raise ValueError
        else:
            values = values.astype(np.int32)
            
        data[col_name] = values
    data['Moyenne'] = np.nan
    data['Eligible'] = np.nan
    data['Decision'] = np.nan
    
    df = pd.DataFrame(data)
    path = f"{fichier}.txt"
    df.to_csv(path, index=False, sep='\t')
    return path

def save_df(df, file=file):
    path = f"{file}.txt"
    df.to_csv(path, index=False, sep='\t')
    return path

def upload_df(file=file):
    try:
        df = pd.read_csv(file, sep='\t')
        return df
    except FileNotFoundError:
        print("Fichier pas trouvé")

In [59]:
fichier = generer_donnees()
print("Fichier créé:", fichier)
df = upload_df(fichier)
display(df)

Fichier créé: data.txt


,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0,16,10,12,12,11,10,10,14,NaN,NaN,NaN
1,1,10,11,13,11,19,11,6,16,NaN,NaN,NaN
2,2,14,7,13,13,11,11,4,13,NaN,NaN,NaN
3,3,14,12,12,13,10,12,5,16,NaN,NaN,NaN
4,4,15,8,14,14,14,14,11,15,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12,6,13,10,10,13,6,14,NaN,NaN,NaN
96,96,9,10,12,11,17,10,8,14,NaN,NaN,NaN
97,97,15,11,13,12,15,11,11,15,NaN,NaN,NaN
98,98,7,11,12,12,18,13,6,15,NaN,NaN,NaN


In [60]:
df.describe().loc[['mean', 'std']]

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
mean,49.500000,10.41000,9.800000,12.370000,12.240000,13.410000,11.700000,8.890000,14.5600,NaN,NaN,NaN
std,29.011492,2.50694,1.874874,1.375507,1.407914,2.774687,1.604917,2.763013,1.2417,NaN,NaN,NaN


# 2 -Calculs des valeurs manquantes en fur et à mesure ou bien au début..

In [61]:
def calcule_MED(S1, S2, S3, S4, S5, Logique, BDD, PS, coeffs=coeffs, seuils=seuils, thresholds2=thresholds):
    moyenne = (S1 * coeffs[0] + S2 * coeffs[1] + S3 * coeffs[2] + S4 * coeffs[3] + S5 * coeffs[4]) / sum(coeffs)
    eligible = Logique > seuils[0] and BDD > seuils[1] and PS > seuils[2]
    if eligible and moyenne > thresholds2[1]:
        decision = 'A'
    elif eligible and thresholds2[0] <= moyenne <= thresholds2[1]:
        decision = 'L'
    else:
        decision = 'R'
    return moyenne, eligible, decision

In [62]:
# c'est un test de calcul de M E D sur un étudiant

if df is not None:
    sample_data = df.iloc[0]
    moyenne, eligible, decision = calcule_MED(sample_data['S1'], sample_data['S2'], sample_data['S3'], sample_data['S4'], sample_data['S5'],
                                              sample_data['Logique'], sample_data['BDD'], sample_data['PS'], coeffs=[1, 1, 1, 1, 1], seuils=[12, 12, 12], thresholds2=[15, 17])

    print("Moyenne:", moyenne)
    print("Eligible:", eligible)
    print("Decision:", decision)
else:
    print("Error: DataFrame is None. Please check if the file exists or if there was an issue reading the file.")



Moyenne: 12.2
Eligible: False
Decision: R


In [63]:
# à partir d'un numéro de candidat, remplir  les données M  E D manquante dans la df.
def update_MED(N_candidat, df):
    row = df[df['N_candidat'] == N_candidat]
    if not row.empty:
        moyenne, eligible, decision = calcule_MED(row['S1'].values[0], row['S2'].values[0], row['S3'].values[0],
                                                   row['S4'].values[0], row['S5'].values[0], row['Logique'].values[0],
                                                   row['BDD'].values[0], row['PS'].values[0])
        df.loc[df['N_candidat'] == N_candidat, 'Moyenne'] = moyenne
        df.loc[df['N_candidat'] == N_candidat, 'Eligible'] = eligible
        df.loc[df['N_candidat'] == N_candidat, 'Decision'] = decision
        
        print(f"MED values for N_candidat {N_candidat} successfully updated in the dataframe")
    else:
        print(f"N_candidat {N_candidat} not found in the dataframe")


In [64]:
update_MED(3,df)


MED values for N_candidat 3 successfully updated in the dataframe


In [65]:
display(df)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0,16,10,12,12,11,10,10,14,NaN,NaN,NaN
1,1,10,11,13,11,19,11,6,16,NaN,NaN,NaN
2,2,14,7,13,13,11,11,4,13,NaN,NaN,NaN
3,3,14,12,12,13,10,12,5,16,12.0,False,R
4,4,15,8,14,14,14,14,11,15,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12,6,13,10,10,13,6,14,NaN,NaN,NaN
96,96,9,10,12,11,17,10,8,14,NaN,NaN,NaN
97,97,15,11,13,12,15,11,11,15,NaN,NaN,NaN
98,98,7,11,12,12,18,13,6,15,NaN,NaN,NaN


In [66]:
save_df(df)

'data.txt'

In [67]:
def compute_all_MED(df):
    for index, row in df.iterrows():
        N_candidat = row['N_candidat']
        moyenne, eligible, decision = calcule_MED(row['S1'], row['S2'], row['S3'], row['S4'], row['S5'],
                                                   row['Logique'], row['BDD'], row['PS'])
        df.loc[index, 'Moyenne'] = moyenne
        df.loc[index, 'Eligible'] = eligible
        df.loc[index, 'Decision'] = decision

    print("dataframe entièrement remplie")



In [68]:
display(df)
compute_all_MED(df)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0,16,10,12,12,11,10,10,14,NaN,NaN,NaN
1,1,10,11,13,11,19,11,6,16,NaN,NaN,NaN
2,2,14,7,13,13,11,11,4,13,NaN,NaN,NaN
3,3,14,12,12,13,10,12,5,16,12.0,False,R
4,4,15,8,14,14,14,14,11,15,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12,6,13,10,10,13,6,14,NaN,NaN,NaN
96,96,9,10,12,11,17,10,8,14,NaN,NaN,NaN
97,97,15,11,13,12,15,11,11,15,NaN,NaN,NaN
98,98,7,11,12,12,18,13,6,15,NaN,NaN,NaN


dataframe entièrement remplie


In [69]:
display(df)
save_df(df)
# now, vas-y regarde le fichier

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0,16,10,12,12,11,10,10,14,12.2,False,R
1,1,10,11,13,11,19,11,6,16,13.2,False,R
2,2,14,7,13,13,11,11,4,13,11.6,False,R
3,3,14,12,12,13,10,12,5,16,12.0,False,R
4,4,15,8,14,14,14,14,11,15,13.0,True,L
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12,6,13,10,10,13,6,14,10.8,False,R
96,96,9,10,12,11,17,10,8,14,12.0,False,R
97,97,15,11,13,12,15,11,11,15,13.4,False,R
98,98,7,11,12,12,18,13,6,15,12.0,False,R


'data.txt'

In [70]:
df.describe().loc[['mean', 'std']]

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne
mean,49.500000,10.41000,9.800000,12.370000,12.240000,13.410000,11.700000,8.890000,14.5600,11.672000
std,29.011492,2.50694,1.874874,1.375507,1.407914,2.774687,1.604917,2.763013,1.2417,0.926608


# 3 - Model training and test

In [71]:
df = upload_df('data.txt')
X = df[['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']]
Y = df['Decision']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred, zero_division=1)

print("Accuracy =", accuracy)
print("Classification Report:\n", report)

Accuracy = 1.0
Classification Report:
               precision    recall  f1-score   support

           L       1.00      1.00      1.00         1
           R       1.00      1.00      1.00        19

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [72]:
scores = cross_val_score(model, X, Y, cv=5)
Y_pred_cv = cross_val_predict(model, X, Y, cv=5)

accuracy = np.mean(scores)
report = classification_report(Y, Y_pred_cv, zero_division=1)

print("Cross-Validation Mean Accuracy:", accuracy)
print("Classification Report:\n", report)


Cross-Validation Mean Accuracy: 0.9099999999999999
Classification Report:
               precision    recall  f1-score   support

           L       1.00      0.10      0.18        10
           R       0.91      1.00      0.95        90

    accuracy                           0.91       100
   macro avg       0.95      0.55      0.57       100
weighted avg       0.92      0.91      0.88       100



In [74]:
def features_of_model(model):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    print("L'importance des features:")
    for f in range(X.shape[1]):
        print(f"    {X.columns[indices[f]]}: {importances[indices[f]]}")


features_of_model(model)
#je veux tester mon git

L'importance des features:
    BDD: 0.20861523999643378
    S1: 0.19110276905031215
    Logique: 0.14685831154160164
    S3: 0.1349750714634743
    S5: 0.10123288852523316
    S2: 0.08808762716890119
    S4: 0.07660549711067033
    PS: 0.05252259514337357
